In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
import csv


In [3]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [4]:
file_name = '/themapark/weather/weather' + cal_std_day(0) + '.json'
tmp = spark.read.json(file_name, encoding='UTF-8')
tmp.show()
tmp.printSchema()

+--------------------+-----------------------------+
|                data|                         meta|
+--------------------+-----------------------------+
|[{0.0, 2022-10-20...|{{강수량, 날짜, 바람, 일교...|
+--------------------+-----------------------------+

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- 강수량: double (nullable = true)
 |    |    |-- 날짜: string (nullable = true)
 |    |    |-- 바람: double (nullable = true)
 |    |    |-- 일교차: double (nullable = true)
 |    |    |-- 지역: string (nullable = true)
 |    |    |-- 최고기온: double (nullable = true)
 |    |    |-- 최대풍속: double (nullable = true)
 |    |    |-- 최저기온: double (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- 강수량: string (nullable = true)
 |    |    |-- 날짜: string (nullable = true)
 |    |    |-- 바람: string (nullable = true)
 |    |    |-- 일교차: string (nullable = true)
 |    |    |-- 지역: string (nullable = tr

In [10]:
data = []
for r1 in tmp.select(tmp.data).toLocalIterator():
    if not r1.data:
        continue
    for r2 in r1.data:
        temp = r2.asDict()
        data.append(temp)

dw = spark.createDataFrame(data)
w_data = dw.select(
                dw.지역.cast(StringType()).alias('LOC')
                ,col('날짜').cast(DateType()).alias('STD_DATE')
                ,col('최고기온').cast(FloatType()).alias('HIGH_TEMP')
                ,col('최저기온').cast(FloatType()).alias('LOW_TEMP')
                ,col('일교차').cast(FloatType()).alias('DIFF_TEMP')
                ,col('강수량').cast(FloatType()).alias('RAIN_AMOUNT')
                ,col('바람').cast(FloatType()).alias('AVG_WIND')
                ,col('최대풍속').cast(FloatType()).alias('HIGH_WIND')
            )

w_data.coalesce(1).\
    write.\
    format("com.databricks.spark.csv").\
    option("header", "true").\
    save("/data/"+cal_std_day(0)+".csv")

In [12]:
type(w_data)

pyspark.sql.dataframe.DataFrame

In [6]:
file_name = '/themapark/dust/dust' + cal_std_day(0) + '.json'
tmp = spark.read.json(file_name, encoding='UTF-8')
tmp.show()
tmp.printSchema()

+--------------------+-----------------------------+
|                data|                         meta|
+--------------------+-----------------------------+
|[{2022-10-20, 24,...|{{날짜, 미세먼지, 지역, 초...|
+--------------------+-----------------------------+

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- 날짜: string (nullable = true)
 |    |    |-- 미세먼지: long (nullable = true)
 |    |    |-- 지역: string (nullable = true)
 |    |    |-- 초미세먼지: long (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- 날짜: string (nullable = true)
 |    |    |-- 미세먼지: string (nullable = true)
 |    |    |-- 지역: string (nullable = true)
 |    |    |-- 초미세먼지: string (nullable = true)
 |    |-- desc: string (nullable = true)



In [12]:
data = []
for r1 in tmp.select(tmp.data).toLocalIterator():
    if not r1.data:
        continue
    for r2 in r1.data:
        temp = r2.asDict()
        data.append(temp)

dd = spark.createDataFrame(data)
w_data = dd.select(
                col('지역').cast(StringType()).alias('LOC')
                ,col('날짜').cast(DateType()).alias('STD_DATE')
                ,col('미세먼지').cast(IntegerType()).alias('PM10')
                ,col('초미세먼지').cast(IntegerType()).alias('PM25')
            )

w_data.show()
w_data.printSchema()


+----+----------+----+----+
| LOC|  STD_DATE|PM10|PM25|
+----+----------+----+----+
|과천|2022-10-20|  24|  12|
|과천|2022-10-21|  45|  32|
|과천|2022-10-22|  45|  27|
|과천|2022-10-23|  33|  17|
|과천|2022-10-24|  27|  11|
|과천|2022-10-25|  26|  11|
|과천|2022-10-26|  35|  17|
|과천|2022-10-27|  30|  15|
|과천|2022-10-28|  29|  15|
|과천|2022-10-29|  21|  11|
|과천|2022-10-30|  25|  14|
|송파|2022-10-20|  34|  14|
|송파|2022-10-21|  45|  32|
|송파|2022-10-22|  41|  26|
|송파|2022-10-23|  34|  17|
|송파|2022-10-24|  29|  13|
|송파|2022-10-25|  28|  15|
|송파|2022-10-26|  32|  16|
|송파|2022-10-27|  29|  11|
|송파|2022-10-28|  27|  11|
+----+----------+----+----+
only showing top 20 rows

root
 |-- LOC: string (nullable = true)
 |-- STD_DATE: date (nullable = true)
 |-- PM10: integer (nullable = true)
 |-- PM25: integer (nullable = true)

